In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import re

import shutil

from functools import partial
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections import raster, utils


pd.set_option('max_columns', None)

/sd2/Moncho/zonalstats_earth_engine/.venv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
def save_location_mapping(row_and_path):
    """
    Common function used to obtain a mapping of polygons to
    the rasters used by the IMAGE. 
    
    This is not part of projections because it assumes IMAGE 
    exists in the global scope.
    """
    row, path = row_and_path
    shape = row['geometry']
    
    subset = raster.find_subset_with_intersection_area(IMAGE, shape)

    if subset.empty:
        with open(path, 'w') as f:
            f.write('')
        return

    subset['id'] = row['id']
    
    subset.to_csv(path, index=False)   

In [3]:
read_path = Path('../Data/Population')
output_path = utils.make_path('../Output/Population/')
partial_path = utils.make_path(output_path / 'partial')
by_country_path = utils.make_path(output_path / 'by_country')

In [4]:
# Validate that all TIF share the same coordinates
x = None
y = None
no_data_value = None

all_tif = sorted(read_path.glob('**/*.tif'))
for tif in all_tif:
    image = utils.read_tif(tif)
    if x is None:
        x = image.x
        y = image.y
        no_data_value = image._FillValue
    else:
        assert np.all(x == image.x) and np.all(y == image.y), tif
        assert no_data_value == image._FillValue
        
base_file = all_tif[0]

# Map raster to polygons

In [6]:
# Read SHP with all polygons (output of PreprocessingLocs.ipynb, also available in Drive)
geo_df = gpd.read_file('../Shapefiles/preprocessed/all_countries_with_eth.shp')

In [7]:
n_processes = 30

IMAGE = utils.read_tif(base_file)

iterator = partial(utils.yield_missing_shapes, save_path=partial_path, prefix=base_file.name[:-4])

if n_processes == 1:
    for row_and_path in tqdm(iterator(geo_df)):
        save_location_mapping(row_and_path)
else:
    with ThreadPoolExecutor(n_processes) as tpe:
        for _ in tqdm(
            tpe.map(save_location_mapping, iterator(geo_df)), 
            total=geo_df.shape[0]
        ):
            pass

 94%|████████████████████████████▏ | 115521/122772 [20:35:15<4:16:39,  2.12s/it]IOStream.flush timed out
IOStream.flush timed out
 94%|███████████████████████████▍ | 115986/122772 [20:53:32<12:11:37,  6.47s/it]IOStream.flush timed out
IOStream.flush timed out
 95%|████████████████████████████▌ | 117061/122772 [21:31:34<7:12:21,  4.54s/it]IOStream.flush timed out
IOStream.flush timed out
 95%|███████████████████████████▋ | 117063/122772 [21:31:58<11:52:22,  7.49s/it]IOStream.flush timed out
IOStream.flush timed out
 97%|████████████████████████████ | 119022/122772 [22:47:02<12:13:01, 11.73s/it]IOStream.flush timed out
IOStream.flush timed out
 98%|████████████████████████████▍| 120282/122772 [23:47:36<10:19:57, 14.94s/it]IOStream.flush timed out
IOStream.flush timed out
100%|███████████████████████████████▉| 122585/122772 [24:08:59<02:12,  1.41it/s]


In [ ]:
utils.union_and_save_portions(read_from=partial_path, save_in=by_country_path)

# Aggregate all files with same raster
As there are multiple TIFs but all of them share the same raster, match the values from these to the preprocessed raster and perform aggregations.

In [ ]:
def aggregate_one(file):
    partial_path = make_partial_path(file.parent.name)
    output_path = partial_path.parent / file.name
    if output_path.exists():
        return file.name
    
    file_path = partial_path.parent / file.name[:-4]
    file_path.mkdir(exist_ok=True)

    IMAGE = utils.read_tif(file)
    increment = raster.get_increment_from_tif(IMAGE)
    
    for df_path in by_country_path.glob('*.feather'):
        subdf_path = file_path / df_path.name
        if subdf_path.exists():
            continue
            
        df = pd.read_feather(df_path)
        pol = utils.get_mock_polygon_from_df(df, increment=increment)
        subdf = raster.merge_df_to_array_by_lat_lon(df, IMAGE, pol)
        if subdf.empty:
            print(df_path.name, 'is empty')
        else:
            subdf.to_feather(subdf_path)
            
    utils.aggregate_feather_splits_and_save(
        input_path=file_path, 
        output_path=output_path, 
        no_data_value=no_data_value
    )
    shutil.rmtree(file_path)
    return file.name
    
n_processes = min(len(all_tif), 10)
print(f"Running with {n_processes} processes")
if n_processes == 1:
    for tif_file in all_tif:
        print(aggregate_one(tif_file))
else:
    with ThreadPoolExecutor(n_processes) as tpe:
        for name in tpe.map(aggregate_one, all_tif):
            print(name)

Running with 10 processes


Grouping: 100%|███████████████████████████| 48136/48136 [35:39<00:00, 22.50it/s]


gpw_v4_population_count_rev11_2000_30_sec.tif


# Join results
Combine all the intermediate results and save a consolidated file

In [ ]:
def read_population_df(file):
    df = utils.robust_read(file)
    year = get_year_from_population_file(file)
    value_name = f'{file.parent.name}_{year}'
    df.rename(columns={'value': value_name}, inplace=True)
    return df

def get_year_from_population_file(file):
    return re.findall(".*_(\d{4})_.*", file.name)[0]

In [ ]:
df = None

for file in tqdm(output_path.glob('**/*.csv')):
    if file.parent.name == 'partial':
        continue
        
    field = read_population_df(file)
    if field.empty:
        continue
    elif df is None:
        df = field
    else:
        df = df.merge(field.drop(columns='intersection_area'), on='id', how='outer')

In [ ]:
df.to_csv(output_path / "population.csv", index=False)